In [1]:
import os
import time
import numpy as np
import renom as rm
from tqdm import tqdm
import matplotlib.pyplot as plt

from renom_img.api.classification.vgg import VGG16, VGG19
from renom_img.api.utility.distributor.distributor import ImageDistributor
from renom_img.api.utility.augmentation import Augmentation
from renom_img.api.utility.augmentation.process import *
from renom_img.api.utility.load import parse_xml_detection
from renom_img.api.utility.misc.display import draw_box
from renom_img.api.utility.misc.dataset import fetch_classification_dataset_caltech101

from renom.cuda import set_cuda_active
set_cuda_active(True)

In [2]:
train_image_path_list, train_label_list, valid_image_path_list, valid_label_list = fetch_classification_dataset_caltech101()

In [3]:
image_caltech101 = "101_ObjectCategories"
class_map = sorted(os.listdir(image_caltech101))[1:]

print("Dataset size")
print("  Train:{}".format(len(train_image_path_list)))
print("  Valid:{}\n".format(len(valid_image_path_list)))

print("Class list")
for i, name in enumerate(class_map):
    print("  {:02d} {}".format(i, name))

Dataset size
  Train:6942
  Valid:1735

Class list
  00 Faces
  01 Faces_easy
  02 Leopards
  03 Motorbikes
  04 accordion
  05 airplanes
  06 anchor
  07 ant
  08 barrel
  09 bass
  10 beaver
  11 binocular
  12 bonsai
  13 brain
  14 brontosaurus
  15 buddha
  16 butterfly
  17 camera
  18 cannon
  19 car_side
  20 ceiling_fan
  21 cellphone
  22 chair
  23 chandelier
  24 cougar_body
  25 cougar_face
  26 crab
  27 crayfish
  28 crocodile
  29 crocodile_head
  30 cup
  31 dalmatian
  32 dollar_bill
  33 dolphin
  34 dragonfly
  35 electric_guitar
  36 elephant
  37 emu
  38 euphonium
  39 ewer
  40 ferry
  41 flamingo
  42 flamingo_head
  43 garfield
  44 gerenuk
  45 gramophone
  46 grand_piano
  47 hawksbill
  48 headphone
  49 hedgehog
  50 helicopter
  51 ibis
  52 inline_skate
  53 joshua_tree
  54 kangaroo
  55 ketch
  56 lamp
  57 laptop
  58 llama
  59 lobster
  60 lotus
  61 mandolin
  62 mayfly
  63 menorah
  64 metronome
  65 minaret
  66 nautilus
  67 octopus
  68 okapi


In [4]:
model = VGG16(class_map, load_pretrained_weight=True, train_whole_network=False, imsize=(124, 124))

/home/grid/renom_env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
train_log, valid_log = model.fit(train_image_path_list, train_label_list, valid_image_path_list, valid_label_list, epoch=10)

Epoch:000 Avg Train Loss:7.478 Avg Valid Loss:3.978: 100%|██████████| 28/28 [00:29<00:00,  5.38it/s]
Epoch:001 Avg Train Loss:2.132 Avg Valid Loss:2.237: 100%|██████████| 28/28 [00:27<00:00,  5.37it/s]
Epoch:002 Avg Train Loss:1.008 Avg Valid Loss:1.812: 100%|██████████| 28/28 [00:28<00:00,  5.23it/s]
Epoch:003 Avg Train Loss:0.544 Avg Valid Loss:1.792: 100%|██████████| 28/28 [00:28<00:00,  5.19it/s]
Epoch:004 Avg Train Loss:0.372 Avg Valid Loss:1.665: 100%|██████████| 28/28 [00:28<00:00,  5.23it/s]
Epoch:005 Avg Train Loss:0.290 Avg Valid Loss:1.595: 100%|██████████| 28/28 [00:28<00:00,  5.27it/s]
Epoch:006 Avg Train Loss:0.201 Avg Valid Loss:1.336: 100%|██████████| 28/28 [00:28<00:00,  5.19it/s]
Epoch:007 Avg Train Loss:0.171 Avg Valid Loss:1.516: 100%|██████████| 28/28 [00:28<00:00,  5.37it/s]
Epoch:008 Avg Train Loss:0.135 Avg Valid Loss:1.503: 100%|██████████| 28/28 [00:28<00:00,  5.25it/s]
Epoch:009 Avg Train Loss:0.094 Avg Valid Loss:1.329: 100%|██████████| 28/28 [00:28<00:00,  

In [7]:
test_image_path_list = valid_image_path_list
test_label_list = valid_label_list

prediction_list = np.array([])
batch_size = 32
for i in range(0, len(test_image_path_list) // batch_size+1):
    prediction = model.predict(test_image_path_list[batch_size*i:batch_size*(i+1)])
    prediction_list = np.append(prediction_list, prediction)


from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(prediction_list, test_label_list))
print(confusion_matrix(prediction_list, test_label_list))

             precision    recall  f1-score   support

        0.0       0.96      0.98      0.97        93
        1.0       0.98      0.93      0.96        70
        2.0       0.95      0.97      0.96        39
        3.0       1.00      0.96      0.98       161
        4.0       0.80      1.00      0.89         8
        5.0       0.97      0.95      0.96       151
        6.0       0.30      0.38      0.33         8
        7.0       0.67      0.60      0.63        10
        8.0       0.71      0.83      0.77         6
        9.0       0.50      0.50      0.50         8
       10.0       0.09      0.25      0.13         4
       11.0       0.60      0.75      0.67         8
       12.0       0.93      0.96      0.94        26
       13.0       0.78      0.95      0.86        19
       14.0       0.50      0.40      0.44         5
       15.0       0.95      0.76      0.84        25
       16.0       0.62      0.53      0.57        15
       17.0       0.92      1.00      0.96   

In [9]:
model_vgg19 = VGG19(class_map, load_pretrained_weight=True, train_whole_network=False, imsize=(124, 124))
model_vgg19.fit(train_image_path_list, train_label_list, valid_image_path_list, valid_label_list, epoch=10)

Epoch:000 Avg Train Loss:7.527 Avg Valid Loss:3.608: 100%|██████████| 28/28 [00:32<00:00,  4.59it/s]
Epoch:001 Avg Train Loss:2.020 Avg Valid Loss:2.294: 100%|██████████| 28/28 [00:32<00:00,  4.60it/s]
Epoch:002 Avg Train Loss:0.914 Avg Valid Loss:1.704: 100%|██████████| 28/28 [00:32<00:00,  4.55it/s]
Epoch:003 Avg Train Loss:0.505 Avg Valid Loss:1.647: 100%|██████████| 28/28 [00:32<00:00,  4.55it/s]
Epoch:004 Avg Train Loss:0.346 Avg Valid Loss:1.415: 100%|██████████| 28/28 [00:32<00:00,  4.46it/s]
Epoch:005 Avg Train Loss:0.265 Avg Valid Loss:1.651: 100%|██████████| 28/28 [00:32<00:00,  4.52it/s]
Epoch:006 Avg Train Loss:0.169 Avg Valid Loss:1.394: 100%|██████████| 28/28 [00:32<00:00,  4.59it/s]
Epoch:007 Avg Train Loss:0.146 Avg Valid Loss:1.338: 100%|██████████| 28/28 [00:32<00:00,  4.62it/s]
Epoch:008 Avg Train Loss:0.118 Avg Valid Loss:1.327: 100%|██████████| 28/28 [00:32<00:00,  4.56it/s]
Epoch:009 Avg Train Loss:0.106 Avg Valid Loss:1.257: 100%|██████████| 28/28 [00:33<00:00,  

([7.5267161207461575,
  2.020151467498289,
  0.9135174447790199,
  0.5045887281725167,
  0.34554245789458443,
  0.26484138121264517,
  0.16935649249540352,
  0.14569166506033973,
  0.11793119136731081,
  0.10648676986008276],
 [3.607955438750131,
  2.2942461304898774,
  1.7035654527800423,
  1.6471741300608431,
  1.4146893178778035,
  1.6512893097741264,
  1.394415014556476,
  1.3383469832312715,
  1.3274780639580317,
  1.256569940596819])

In [11]:
test_image_path_list = valid_image_path_list
test_label_list = valid_label_list

prediction_list = np.array([])
batch_size = 32
for i in range(0, len(test_image_path_list) // batch_size+1):
    prediction = model_vgg19.predict(test_image_path_list[batch_size*i:batch_size*(i+1)])
    prediction_list = np.append(prediction_list, prediction)


from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(prediction_list, test_label_list))
print(confusion_matrix(prediction_list, test_label_list))

             precision    recall  f1-score   support

        0.0       0.97      0.98      0.97        94
        1.0       0.97      0.98      0.98        65
        2.0       0.97      0.93      0.95        42
        3.0       0.99      0.95      0.97       161
        4.0       0.90      0.90      0.90        10
        5.0       0.97      0.96      0.96       148
        6.0       0.40      0.50      0.44         8
        7.0       0.67      1.00      0.80         6
        8.0       0.86      0.75      0.80         8
        9.0       0.50      0.67      0.57         6
       10.0       0.36      0.44      0.40         9
       11.0       0.50      0.83      0.62         6
       12.0       0.78      0.75      0.76        28
       13.0       0.87      0.77      0.82        26
       14.0       0.50      0.33      0.40         6
       15.0       0.90      0.75      0.82        24
       16.0       0.69      0.41      0.51        22
       17.0       1.00      0.92      0.96   